# GPX elevation
Notebook by [Florian Neukirchen](https://www.riannek.de/). 
For detailed explanation see my blog: https://www.riannek.de/2022/elevation-to-gps-track-python/

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import elevation
import rasterio

In [ ]:
track_in = 'gpx/2021-05-22-163014.gpx' 
track_out = 'test.gpx'

dem_file = '2021-05-22-163014.tif' 
demfolder = "dem" # Where to save the DEM

# Note: this uses GDAL under the hood
# https://gdal.org/drivers/vector/gpx.html
gdf = gpd.read_file(track_in, layer='track_points')

# GDAL returns a lot of useless columns with "None" values
gdf.dropna(axis=1, inplace=True)

# This is important if we want to save as GPX
gdf['time'] = pd.to_datetime(gdf['time'])

gdf.head()

In [ ]:
gdf.drop(columns='ele', inplace=True)
gdf.head()

In [ ]:
# Get the bounds of the track
minx, miny, maxx, maxy = gdf.dissolve().bounds.loc[0]

# You might want to add some margin if you want to use the DEM for other tracks/tasks
minx, miny, maxx, maxy = bounds  = minx - .05, miny - .05, maxx + .05, maxy + .05
bounds

## Get the DEM

In [ ]:
# make sure the dem folder does exist
# (otherwise the module elevation fails)
if not os.path.exists(demfolder):
    os.mkdir(demfolder)
    
# the module elevation does not work with relative paths
dem_file = os.path.join(os.getcwd(), demfolder, dem_file)

In [ ]:
# Download DEM 
elevation.clip(bounds=bounds, output=dem_file)

# Clean temporary files
elevation.clean()

In [ ]:
# Open the DEM 
dem_data = rasterio.open(dem_file)

In [ ]:
# The CRS is EPSG 4326 = WGS84
# (The same CRS as used by GPS devices)
dem_data.crs

In [ ]:
dem_data.width

In [ ]:
dem_data.height

In [ ]:
dem_data.bounds

In [ ]:
# It has one channel ...
dem_data.count

In [ ]:
# ... and we want this channel as array
dem_array = dem_data.read(1)

## Sample the DEM
(using my alternative approach in 03_runkeeper)

All the magic is happening here: dem_data.index() takes a set of coordinates and returns the index values of our array. We can use these index values to get the value from the array. Note: GPX uses the tag `<ele></ele>` for elevation.

In [ ]:
gdf['ele'] = dem_array[dem_data.index(gdf['geometry'].x , gdf['geometry'].y)]

# Note that we get ele values as int. 
# To save as GPX, ele must be as float.
gdf['ele'] = gdf['ele'].astype(float)

gdf.head()

## Save as GPX
Note: This fails if the file already exists!

In [ ]:
gdf.to_file(track_out, 'GPX', layer='track_points')

## Extra: Have a look at the DEM

In [ ]:
import seaborn as sns
# Histplot of elevations
sns.displot(dem_array.ravel())

In [ ]:
# Save figure
sns.displot(dem_array.ravel()).savefig('dem-hist.png')

In [ ]:
import matplotlib.pyplot as plt

vmin = dem_array.min()
vmax = dem_array.max()
extent = minx, maxx, miny, maxy

fig, ax = plt.subplots()
cax = plt.imshow(dem_array, extent=extent, 
                  cmap='Spectral_r', vmin=vmin, vmax=vmax)
fig.colorbar(cax, ax=ax)
gdf.plot(ax=ax, markersize=1, zorder=11)

In [ ]:
fig.savefig('gersb-dem.png')